In [18]:
### Importing Packages
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm
import statsmodels.tools

In [19]:
### Reading in the file saved from the EDA notebook (after cleaning too)
# Deleting the 'Unnamed' column again
df = pd.read_csv('eda_cleaned_data.csv')
df.drop(columns = ['Unnamed: 0'], inplace = True)

In [20]:
### Reducing the price to 150,000-250,000 for this model
df = df[df['price'] >= 250000]
df = df[df['price'] < 350000]

In [21]:
### Reducing the number of bedrooms to just 2,3 and 4 for this model
del_beds = [1,5, 6, 7, 8, 9]
df = df[~df['bedroom'].isin(del_beds)]

<font size="6"><font color='orange'>Prepping categorical data for modelling (Feature engineering)</font>

In [22]:
### Feature Engineering Function
# Non-numeric columns ('house_type' and 'postcode') are One-Hot Encoded


def feature_eng(df):
    ''' Feature Engineer Function - One Hot encoding of house_type and postcode'''
    df = pd.get_dummies(df, columns = ['house_type'], drop_first = True, prefix = 'type')
    df.drop(columns = ['property_no'], inplace = True)
    df.drop(columns = ['postcode'], inplace = True)
    df.drop(columns = ['longitude'], inplace = True)
    df.drop(columns = ['latitude'], inplace = True)

    df = sm.add_constant(df)

    return df

<font size="6"><font color='orange'>Model</font>

In [23]:
### Train / Test Splitting
feature_cols = list(df.columns)        ## Columns from Dataframe set to Feature Columns for Model Testing
feature_cols.remove('price')            ## Removing the dependent variable, price

X = df[feature_cols]
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)  ## Splitting Dataframe data into separate train and test dataframes



In [24]:
## Apply feature engineering function X_Train Dataframe
X_train = feature_eng(X_train)

In [25]:
## Scale mileage and age_log using MinMax Scaler
scaler = StandardScaler()
X_train[['bedroom']] = scaler.fit_transform(X_train[['bedroom']])          ## Fitting the model to X-train data
scaler.scale_       ## Scaling factors to be implemented in the function

array([0.53942711])

In [26]:
## Scale mileage and age_log using MinMax Scaler
scaler = StandardScaler()
X_train[['distance']] = scaler.fit_transform(X_train[['distance']])         ## Fitting the model to X-train data
scaler.scale_       ## Scaling factors to be implemented in the function

array([4.85907366])

In [27]:
# Checking what columns to put into the feature_cols
X_train.columns

Index(['const', 'bedroom', 'distance', 'type_Semi-Detached', 'type_Terraced'], dtype='object')

In [28]:
# Craeting a constant column (this is a failsafe as sometimes the const column wasn't auto-created)
X_train['const'] = 1

In [29]:
### Linear Regression
feature_cols = ['const','bedroom', 'distance', 'type_Semi-Detached', 'type_Terraced']


                ## Final Features to be used in the model


lin_reg = sm.OLS(y_train, X_train[feature_cols].astype(float))          ## Stats Model Linear Regression
results = lin_reg.fit()
print(results.summary())


X_train['y_pred'] = results.predict(X_train[feature_cols])
X_train['y_pred'] = X_train['y_pred'].astype('float')
rmse = statsmodels.tools.eval_measures.rmse(y_train, X_train['y_pred']) ## RMSE of the Model

print('\n', 'Rmse: ', rmse)

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.212
Model:                            OLS   Adj. R-squared:                  0.203
Method:                 Least Squares   F-statistic:                     23.84
Date:                Fri, 23 Dec 2022   Prob (F-statistic):           1.78e-17
Time:                        13:19:52   Log-Likelihood:                -4146.9
No. Observations:                 359   AIC:                             8304.
Df Residuals:                     354   BIC:                             8323.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const               2.976e+05   1875

In [30]:
### VIF
def checkVIF(X):
    '''Function to calculate the VIF of a dataframe '''
    vif = pd.DataFrame()                ## Initialising an empty dataframe
    vif['Features'] = feature_cols      ## The features used in training
    vif['VIF'] = [variance_inflation_factor(X[feature_cols].values, i) for i in range(X[feature_cols].shape[1])]
    vif['VIF'] = round(vif['VIF'], 2)   ## Round VIF to 2 decimal places for readability
    vif = vif.sort_values(by = "VIF", ascending = False)    ## Sorting VIF values in descending order to observe feature VIF scores
    return vif
checkVIF(X_train)   ## If VIF >= 5, test to see if the removal of said feature reduces the multi-co-linearity

,Features,VIF
0,const,1.97
3,type_Semi-Detached,1.21
1,bedroom,1.17
4,type_Terraced,1.12
2,distance,1.00


<font size="6"><font color='orange'>Testing the model</font>

In [31]:
## Apply feature engineering function X_Train Dataframe
X_test = feature_eng(X_test)

In [32]:
## Scale mileage and age_log using MinMax Scaler
scaler = StandardScaler()
X_test[['bedroom']] = scaler.fit_transform(X_test[['bedroom']])  ## Fitting the model to X-train data
scaler.scale_  ## Scaling factors to be implemented in the function

array([0.54160256])

In [33]:
## Scale mileage and age_log using MinMax Scaler
scaler = StandardScaler()
X_test[['distance']] = scaler.fit_transform(X_test[['distance']])  ## Fitting the model to X-train data
scaler.scale_  ## Scaling factors to be implemented in the function

array([5.03481504])

In [34]:
X_train['const'] = 1
### Linear Regression (with postcode)
feature_cols = ['const', 'bedroom', 'distance', 'type_Semi-Detached', 'type_Terraced']

## Final Features to be used in the model


lin_reg = sm.OLS(y_test, X_test[feature_cols].astype(float))  ## Stats Model Linear Regression
results = lin_reg.fit()
print(results.summary())

X_test['y_pred'] = results.predict(X_test[feature_cols])
X_test['y_pred'] = X_test['y_pred'].astype('float')
rmse = statsmodels.tools.eval_measures.rmse(y_test, X_test['y_pred'])  ## RMSE of the Model

print('\n', 'Rmse: ', rmse)

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.168
Model:                            OLS   Adj. R-squared:                  0.129
Method:                 Least Squares   F-statistic:                     4.302
Date:                Fri, 23 Dec 2022   Prob (F-statistic):            0.00323
Time:                        13:19:52   Log-Likelihood:                -1047.0
No. Observations:                  90   AIC:                             2104.
Df Residuals:                      85   BIC:                             2117.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const               2.893e+05   4146